<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Retrieval Augmented Generation (RAG)
## Overview  
In this exercise, we will delve into document embedding techniques. You will learn how to build a powerful Retriever using the Solar Embedding API to generate high-quality embeddings. By the end of this tutorial, you will also have a solid understanding of how to integrate a Retriever with an LLM Chain, enabling more efficient and accurate information retrieval.

In [2]:
!pip3 install -qU python-dotenv PyPDF2 langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb python-dotenv

In [1]:
from langchain_upstage import UpstageLayoutAnalysisLoader #pdf 파일을 raw text(등등)로 바꿔주는 애

UPSTAGE_API_KEY = "up_svEvm8FOA6g9OILcC8oqs9bg4RSB7"
layzer = UpstageLayoutAnalysisLoader(api_key=UPSTAGE_API_KEY,file_path="ewha.pdf", output_type="text")

docs = layzer.load()  # or layzer.lazy_load()

/home/aix7101/anaconda3/envs/nlp/lib/python3.10/site-packages/langchain_upstage/layout_analysis.py:135: UserWarning: UpstageLayoutAnalysisLoader is deprecated.Please use langchain_upstage.document_parse.UpstageDocumentParseLoader instead.
  warnings.warn(
/home/aix7101/anaconda3/envs/nlp/lib/python3.10/site-packages/langchain_upstage/layout_analysis_parsers.py:160: UserWarning: UpstageLayoutAnalysisParser is deprecated.Please use langchain_upstage.document_parse_parsers.UpstageDocumentParseParser instead.
  warnings.warn(


In [4]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

# 2. Split
text_splitter = RecursiveCharacterTextSplitter.from_language( #우리가 원하는 size로 split하기
    chunk_size=1000, chunk_overlap=100, language=Language.HTML #html은 걍 defult값으로 넣어둠
    # num of token == chunk_size (hyperparam)
    # split 하는데 겹치는 부분도 만듦! oh!
    # 
)
splits = text_splitter.split_documents(docs)
print("Splits:", len(splits))

Splits: 53


In [7]:
print(splits[0])

page_content='이화여자대학교 학칙1946. 8. 15. 제정
2017. 8. 16. 개정제1장 총칙제1조(목적) 본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과
그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에
공헌할 수 있는 지도여성을 양성함을 목적으로 한다.제2조(명칭) 본교는 이화여자대학교라 부른다.
제3조(위치) 본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)제2장 편제제4조(대학 및 대학원) ① 본교에는 다음 각 호의 대학을 둔다.1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범
대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각
대학”이라 한다) (개정 2016.6.16.)
2. 호크마(HOKMA)교양대학
② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대
학원, 디자인대학원, 사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보
건융합대학원, 임상치의학대학원, 외국어교육특수대학원을 둔다(이하 “각 대학원”이라 한다).
(개정 2016.6.16., 2017.5.15.)
[전문개정 2015.11.27.]제5조(학부․학과․전공 및 정원) ① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은 별표
1과 같다. (개정 2015.5.8., 2016.2.16., 2016.2.26., 2016.5.19., 2017.5.4., 2017.5.15.)② 모집단위별 입학정원의 일부는 입학전형에 따라 2개 이상의 모집단위를 통합하여 모집
할 수 있다. (개정 1999.2.9., 2017.5.15.)
③ 제2항에 따라 통합된 모집단위로 입학한 학생과 대학 또는 학부 등 광역화된 모집단위
로 입학한 학생에 대하여는 일정한 학기와 학점을 이수한 후에 총장의 승인을 얻어 이수할
전공을 결정하게 하되 이에 필요한 사항은 총장이 따로 정한다

In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = UPSTAGE_API_KEY)

prompt_template = PromptTemplate.from_template( #여기가 prompt engineering !
    """
    Please provide most correct answer from the following context. 
    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {question}
    ---
    Context: {context}
    """
)
chain = prompt_template | llm

In [13]:

response = chain.invoke({"question": "인공지능대학에 대해서 알려줘.", "context": splits[0]})
print(response.content)

인공지능대학에 대한 정보는 해당 문맥에 없습니다.


In [14]:
from langchain_upstage import UpstageEmbeddings # 우리 doc에 대한 embedding을 알 수 잇음
# 질문과 관련된 문서를 찾는 방법? -> 과제
# 문서를 몇 개로 쪼갤지 실험 .. -> 과제

upstage_embeddings = UpstageEmbeddings(api_key=UPSTAGE_API_KEY, model="solar-embedding-1-large")

embedded_query = upstage_embeddings.embed_query(splits[0])
print(embedded_query)

[-0.009063720703125, -0.01593017578125, -0.014404296875, -0.02423095703125, 0.002948760986328125, 0.0042266845703125, 0.0014295578002929688, -0.0175018310546875, -0.015228271484375, 0.044921875, -0.019989013671875, 0.01495361328125, -0.021575927734375, 0.005619049072265625, -0.015106201171875, -0.0177154541015625, 0.0029449462890625, 0.00862884521484375, -0.0271453857421875, -0.0129852294921875, 0.0122222900390625, 0.00901031494140625, -0.011016845703125, -0.015228271484375, -0.0250701904296875, 0.00568389892578125, -0.01401519775390625, -0.02044677734375, 0.0311737060546875, 0.01137542724609375, 0.01666259765625, -0.005950927734375, -0.0139923095703125, 0.022003173828125, -0.0010404586791992188, 0.0007719993591308594, -0.021728515625, -0.00856781005859375, -0.00078582763671875, 0.03363037109375, -0.026702880859375, 0.023895263671875, -0.046539306640625, -0.0008821487426757812, -0.007076263427734375, -0.01953125, -0.01024627685546875, -0.00286865234375, -0.00399017333984375, 0.01416778

In [30]:
# Additional Contents : https://wikidocs.net/253106